# Setup & Test - DINO + CIFAR-100

This notebook:
1. Tests the refactored `src` modules
2. Loads **DINO ViT-S/16** using `src.model`

3. Loads **CIFAR-100** using `src.data`4. Verifies everything works

## 1. Install Required Dependencies

In [ ]:
# Install required packages (run if needed)
#!pip install -q -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
import os
import torch
import torchvision

from src.utils import set_seed, get_device, count_parameters

from src.data import load_cifar100, create_dataloader, partition_iid, partition_non_iid

from src.model import DINOClassifier, load_dino_backbone

# Set seed for reproducibility

print(f"PyTorch version: {torch.__version__}")

print(f"Torchvision version: {torchvision.__version__}")

device = get_device()

print(f"Device: {device}")

PyTorch version: 2.9.0+cpu
Torchvision version: 0.24.0+cpu
CUDA available: False


## 2. Download DINO ViT-S/16 Model

Loading the pretrained DINO ViT-S/16 model from the official Facebook Research GitHub repository:
https://github.com/facebookresearch/dino

In [ ]:
# Load DINO ViT-S/16 using refactored model.py
print("Loading DINO ViT-S/16 model...")

model = DINOClassifier(model_name='dino_vits16', num_classes=100, device=device)
model.eval()

print("\nDINO Classifier loaded successfully!")
print(f"Backbone frozen: {model.freeze_backbone}")
print(f"Output classes: 100")

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main



DINO ViT-S/16 model loaded successfully!
Model architecture:
VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((384,),

In [ ]:
# Model information using refactored utils
total_params = count_parameters(model, trainable_only=False)
trainable_params = count_parameters(model, trainable_only=True)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Trainable %: {100 * trainable_params / total_params:.2f}%")

Total parameters: 21,665,664
Trainable parameters: 21,665,664


## 3. Download CIFAR-100 Dataset

Downloading the CIFAR-100 dataset using torchvision. The dataset contains 60,000 32x32 color images in 100 classes.

In [ ]:
# Define data directory
data_dir = './data'
print(f"Data directory: {os.path.abspath(data_dir)}")

Data will be saved to: /content/data


In [ ]:
# Load CIFAR-100 using refactored data.py (with DINO transforms)
print("Loading CIFAR-100 dataset...")
train_dataset, test_dataset = load_cifar100(data_dir=data_dir, image_size=224)

print(f"Training set: {len(train_dataset)} images")
print(f"Test set: {len(test_dataset)} images")
print(f"Image size after transform: 224x224 (DINO input size)")

100%|██████████| 169M/169M [00:03<00:00, 49.1MB/s] 


Training set size: 50000 images

Test set size: 10000 images


In [ ]:
# Display dataset information
print("\n" + "="*50)
print("CIFAR-100 Dataset Information")
print("="*50)
print(f"Number of classes: {len(train_dataset.classes)}")
print(f"Original size: 32x32x3")
print(f"Transformed size: 224x224x3 (for DINO)")
print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"\nSample classes: {train_dataset.classes[:10]}...")


CIFAR-100 Dataset Information
Number of classes: 100
Image size: 32x32x3
Training samples: 50000
Test samples: 10000

Sample classes: ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle']...


## 4. Test Forward Pass

In [ ]:
# Test model with a batch
test_loader = create_dataloader(test_dataset, batch_size=8, shuffle=False, num_workers=0)
images, labels = next(iter(test_loader))

print(f"Batch shape: {images.shape}")
print(f"Labels: {labels}")

# Forward pass
model.to(device)
images = images.to(device)
with torch.no_grad():
    outputs = model(images)

print(f"\nOutput shape: {outputs.shape}")
print(f"Output range: [{outputs.min():.2f}, {outputs.max():.2f}]")
print("\n✓ Forward pass successful!")

## 5. Test FL Partitioning (M1)

In [ ]:
# Test IID partitioning
print("Testing IID partitioning...")
num_clients = 10
client_datasets_iid = partition_iid(train_dataset, num_clients=num_clients, seed=42)

print(f"Number of clients: {num_clients}")
for i in range(min(3, num_clients)):
    print(f"Client {i}: {len(client_datasets_iid[i])} samples")

# Test non-IID partitioning
print("\nTesting non-IID partitioning...")
num_classes_per_client = 10
client_datasets_noniid = partition_non_iid(
    train_dataset, 
    num_clients=num_clients, 
    num_classes_per_client=num_classes_per_client,
    seed=42
)

for i in range(min(3, num_clients)):
    # Get labels for this client
    subset = client_datasets_noniid[i]
    client_labels = [train_dataset.targets[idx] for idx in subset.indices]
    unique_classes = len(set(client_labels))
    print(f"Client {i}: {len(subset)} samples, {unique_classes} unique classes")

print("\n✓ FL partitioning works!")

## 6. Summary

Setup completed successfully!

In [ ]:
print("\n" + "="*50)
print("Setup Complete!")
print("="*50)
print("\n✓ All refactored modules working:")
print("  - src.utils: set_seed, get_device, count_parameters")
print("  - src.data: load_cifar100, create_dataloader, partitioning")
print("  - src.model: DINOClassifier")

print("\n✓ DINO model loaded and tested")print("\nReady for M3 (Central Baseline) and M4 (FedAvg)!")

print("✓ CIFAR-100 data loaded with transforms")print("✓ FL partitioning (IID/non-IID) verified")